<p style="font-size:150%">Зачет (решение)</p>


In [7]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, text
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """
    :return: Объект соединения или None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

def create_table(conn, sql_statement):
    try:
        crs = conn.cursor()
        crs.execute('DROP TABLE IF EXISTS BikeShare;')
        crs.execute(sql_statement)
    except Error as e:
        print(e)

create_table_project="""
    CREATE TABLE BikeShare (
	ride_id text PRIMARY KEY,
	rideable_type text,
	started_at text,
	ended_at text,
	start_station_name text,
	start_station_id integer,
	end_station_name text,
	end_station_id integer,
	start_lat real,
	start_lng real,
	end_lat real,
	end_lng real,
	member_casual text,
	duration integer);
"""

def main():
    database = "BikeShare.sqlite"
    conn = create_connection(database)
    if conn is not None:
        create_table(conn, create_table_project)
    else:
        print("Ошибка! Не могу создать соединение с БД")

    engine = create_engine('sqlite+pysqlite:///BikeShare.sqlite')
    metadata = MetaData()
    metadata.reflect(bind=engine)

    BikeShare=Table("BikeShare", metadata, autoload=True, autoload_with="engine")

main()

In [8]:
database = "BikeShare.sqlite"
conn = create_connection(database)
data = pd.read_csv('BikeShare_19_20.csv')
data.to_sql('BikeShare', conn, if_exists='append', index=False)

engine = create_engine('sqlite+pysqlite:///BikeShare.sqlite')
metadata = MetaData()
metadata.reflect(bind=engine)

BikeShare=Table("BikeShare", metadata, autoload=True, autoload_with="engine")

In [10]:
s = """
SELECT A.*, A.duration - A.max_duration
FROM (
    SELECT BikeShare.start_station_name, BikeShare.end_station_name, BikeShare.duration, max(BikeShare.duration) OVER (PARTITION BY BikeShare.start_station_id, BikeShare.end_station_id) AS max_duration
    FROM BikeShare) AS A
ORDER BY A.start_station_name, A.end_station_name;
"""


with engine.connect() as conn:
    result = conn.execute(s).fetchall()
    print(len(result))
    for row in result:
        print(row)


457
('10th St & Constitution Ave NW', '10th St & Constitution Ave NW', 3282, 190310, -187028)
('10th St & Constitution Ave NW', '10th St & Constitution Ave NW', 7205, 190310, -183105)
('10th St & Constitution Ave NW', '10th St & Constitution Ave NW', 3316, 190310, -186994)
('10th St & Constitution Ave NW', '10th St & Constitution Ave NW', 4261, 190310, -186049)
('10th St & Constitution Ave NW', '10th St & Constitution Ave NW', 190310, 190310, 0)
('10th St & Constitution Ave NW', '10th St & Constitution Ave NW', 7181, 190310, -183129)
("10th St & L'Enfant Plaza SW", '15th St & Pennsylvania Ave NW', 4887, 4995, -108)
("10th St & L'Enfant Plaza SW", '15th St & Pennsylvania Ave NW', 719, 4995, -4276)
("10th St & L'Enfant Plaza SW", '15th St & Pennsylvania Ave NW', 911, 4995, -4084)
("10th St & L'Enfant Plaza SW", '15th St & Pennsylvania Ave NW', 1722, 4995, -3273)
("10th St & L'Enfant Plaza SW", '15th St & Pennsylvania Ave NW', 756, 4995, -4239)
("10th St & L'Enfant Plaza SW", '15th St & P